## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,city_wind,Current Description
0,0,Katsuura,JP,2022-06-08 19:29:48,35.1333,140.3000,63.84,90,94,9.71,overcast clouds
1,1,Bluff,NZ,2022-06-08 19:29:49,-46.6000,168.3333,39.79,86,81,11.10,broken clouds
2,2,Saint Anthony,US,2022-06-08 19:29:51,45.0205,-93.2180,77.97,32,75,12.66,broken clouds
3,3,Lebu,CL,2022-06-08 19:29:52,-37.6167,-73.6500,54.54,88,83,10.47,broken clouds
4,4,Cherskiy,RU,2022-06-08 19:29:53,68.7500,161.3000,54.91,57,66,12.46,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
print("What is your minimum temperature preference?")
min_temp = float(input("min temp: ")) #55 deg

print("What is your maximum temperature preference?")
max_temp = float(input("max temp: ")) #85 deg

What is your minimum temperature preference?
min temp: 55
What is your maximum temperature preference?
max temp: 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
user_pref_city_df = city_data_df.loc[(city_data_df["Max Temp"] > min_temp) & (city_data_df["Max Temp"] < max_temp)]
user_pref_city_df.head()

,City ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,city_wind,Current Description
0,0,Katsuura,JP,2022-06-08 19:29:48,35.1333,140.3000,63.84,90,94,9.71,overcast clouds
2,2,Saint Anthony,US,2022-06-08 19:29:51,45.0205,-93.2180,77.97,32,75,12.66,broken clouds
5,5,Cabo San Lucas,MX,2022-06-08 19:29:54,22.8909,-109.9124,83.61,57,0,9.22,clear sky
6,6,Illapel,CL,2022-06-08 19:29:54,-31.6308,-71.1653,65.44,53,26,6.55,scattered clouds
9,9,Rikitea,PF,2022-06-08 19:29:57,-23.1203,-134.9692,72.72,64,96,14.09,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
user_pref_city_df.isnull().sum()

City ID                0
City                   0
Country                4
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
city_wind              0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = user_pref_city_df.dropna()
clean_df.head()

,City ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,city_wind,Current Description
0,0,Katsuura,JP,2022-06-08 19:29:48,35.1333,140.3000,63.84,90,94,9.71,overcast clouds
2,2,Saint Anthony,US,2022-06-08 19:29:51,45.0205,-93.2180,77.97,32,75,12.66,broken clouds
5,5,Cabo San Lucas,MX,2022-06-08 19:29:54,22.8909,-109.9124,83.61,57,0,9.22,clear sky
6,6,Illapel,CL,2022-06-08 19:29:54,-31.6308,-71.1653,65.44,53,26,6.55,scattered clouds
9,9,Rikitea,PF,2022-06-08 19:29:57,-23.1203,-134.9692,72.72,64,96,14.09,overcast clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Katsuura,JP,63.84,overcast clouds,35.1333,140.3000,
2,Saint Anthony,US,77.97,broken clouds,45.0205,-93.2180,
5,Cabo San Lucas,MX,83.61,clear sky,22.8909,-109.9124,
6,Illapel,CL,65.44,scattered clouds,-31.6308,-71.1653,
9,Rikitea,PF,72.72,overcast clouds,-23.1203,-134.9692,
10,Carnarvon,AU,64.47,light rain,-24.8667,113.6333,
14,Yellowknife,CA,70.32,few clouds,62.4560,-114.3525,
17,Kloulklubed,PW,73.90,overcast clouds,7.0419,134.2556,
21,Hermanus,ZA,63.61,scattered clouds,-34.4187,19.2345,
22,Port Elizabeth,ZA,68.31,clear sky,-33.9180,25.5701,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        print("Hotel found!")
    except (IndexError):
        print("Cannot find hotel. Skipping...")
        pass
    

Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Cannot find hotel. Skipping...
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Cannot find hotel. Skipping...
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Ho

In [9]:
# 7. Drop the rows where there is no Hotel Name.
c = hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df = c.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Katsuura,JP,63.84,overcast clouds,35.1333,140.3000,Katsuura Hotel Mikazuki
2,Saint Anthony,US,77.97,broken clouds,45.0205,-93.2180,Holiday Inn Express Roseville-St. Paul
5,Cabo San Lucas,MX,83.61,clear sky,22.8909,-109.9124,Vista Roof Restaurant
6,Illapel,CL,65.44,scattered clouds,-31.6308,-71.1653,Hotel Londres
9,Rikitea,PF,72.72,overcast clouds,-23.1203,-134.9692,People ThankYou


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.4)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))